In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.cloud import storage
from pyspark.sql.functions import col, mean, stddev, expr, date_format, count, avg
from pyspark.sql import DataFrame

In [2]:
# Create SparkContext
sc = SparkContext.getOrCreate()

# Set the log level to ERROR to suppress INFO messages
sc.setLogLevel("ERROR")

In [3]:
#fix the formating of the shows, so they don't overlap.
def hscroll(activate=True):
  """activate/deactivate horizontal scrolling for wide output cells"""
  from IPython.display import display, HTML
  style = ('pre-wrap','pre')[activate] # select white-space style
  display(HTML("<style>pre {white-space: %s !important}</style>" % style))
hscroll()

In [4]:
#removes rows were outliers are present in the selected column
def remove_outliers_in_columns(df, column_name, threshold=2):
    # Calculate mean and standard deviation
    mean_val = df.select(mean(col(column_name))).collect()[0][0]
    std_val = df.select(stddev(col(column_name))).collect()[0][0]

    # Calculate lower and upper bounds
    lower_bound = mean_val - threshold * std_val
    upper_bound = mean_val + threshold * std_val

    # Filter out rows outside the threshold
    filtered_df = df.filter((col(column_name) >= lower_bound) & (col(column_name) <= upper_bound))

    return filtered_df

In [18]:
# Create SparkSession
spark = SparkSession.builder.getOrCreate()

# Initialize DataFrames to hold the joined results
tp_pc_borough_time_df = None
#trips_over_time_df = None

#import the boroughs_sdf dataframe
boroughs_sdf = spark.read.csv('gs://my-bigdata-project-cm/external_data/taxi_zone_lookup.csv', inferSchema=True, header=True)
columns_to_drop = ['Zone','service_zone']
boroughs_sdf = boroughs_sdf.drop(*columns_to_drop)

# Create a client object that points to GCS
storage_client = storage.Client()

# Get a list of the 'blobs' (objects or files) in the bucket
blobs = storage_client.list_blobs('my-bigdata-project-cm', prefix="cleaned/")

#run through the my-bigdata-project-cm bucket.
for blob in blobs:
    file_path = f'gs://my-bigdata-project-cm/{blob.name}'
    
    #exscluding the file title.
    if not blob.name.endswith('.parquet') or blob.name == 'cleaned/':
        print(f"Skipping file {blob.name}")
        continue
        
    try:
        # Read Parquet file from Google Cloud Storage
        sdf = spark.read.parquet(file_path)
        #sdf = sdf.sample(False, 0.2)
        
        #drop unneeded columns
        columns_to_drop = ['store_and_fwd_flag', 'VendorID']
        sdf = sdf.drop(*columns_to_drop)
        
        # Sum up the numerical values of the trip fee columns
        sdf = sdf.withColumn("total_amount", expr("total_amount + congestion_surcharge + airport_fee"))
            
        # Drop the columns related to the trip fee, as they are now redundant
        columns_to_drop = ["fare_amount","extra","mta_tax","tolls_amount","improvement_surcharge","congestion_surcharge","airport_fee"]
        sdf = sdf.drop(*columns_to_drop)
        
        
        print(f"Processing {blob.name}:")
        
    except Exception as e:
        print(f"An error occurred on {blob.name}:", str(e))
        continue

    #removes outliers from the passenger_count column
    sdf = remove_outliers_in_columns(sdf, 'passenger_count')
    #removes outliers from the trip_distance column
    sdf = remove_outliers_in_columns(sdf, 'trip_distance')
    #removes outliers from the tip_amount column
    sdf = remove_outliers_in_columns(sdf, 'tip_amount')

    # Calculate the absolute difference in months between pickup and dropoff datetime
    sdf = sdf.withColumn("pickup_dropoff_month_diff", expr("abs(months_between(tpep_dropoff_datetime, tpep_pickup_datetime))"))
    # Filter out rows where the absolute difference is more than 2 months
    sdf = sdf.filter(col("pickup_dropoff_month_diff") <= 2)
    sdf = sdf.drop('pickup_dropoff_month_diff')
    
    # Joining boroughs_sdf with boroughs_sdf based on PULocationID
    sdf = sdf.join(boroughs_sdf, sdf['PULocationID'] == boroughs_sdf['LocationID'], 'left') \
        .drop('LocationID') \
        .withColumnRenamed('Borough', 'Pickup_Borough')

    # Joining boroughs_sdf with trips_with_pickup_borough based on DOLocationID
    #sdf = sdf.join(boroughs_sdf, sdf['DOLocationID'] == boroughs_sdf['LocationID'], 'left') \
    #    .drop('LocationID') \
    #    .withColumnRenamed('Borough', 'Dropoff_Borough')
    

    # Creating new columns for pickup_month, pickup_year, and pickup_hour using date_format
    sdf = sdf.withColumn('pickup_month', date_format('tpep_pickup_datetime', 'MM'))
    sdf = sdf.withColumn('pickup_year', date_format('tpep_pickup_datetime', 'yyyy'))
    sdf = sdf.withColumn('pickup_hour', date_format('tpep_pickup_datetime', 'HH'))

    # Aggregations remain the same as before
    tp_pc_borough_time = sdf.groupBy('Pickup_Borough', 'pickup_hour', 'pickup_year', 'pickup_month').agg(avg('passenger_count'), avg('tip_amount'), avg('total_amount'),count('*'))
    #trips_over_time = sdf.groupBy('pickup_hour', 'pickup_year', 'pickup_month').agg(count('*'))

    # Combine the DataFrames each loop
    if tp_pc_borough_time_df is not None:
        tp_pc_borough_time_df = tp_pc_borough_time_df.union(tp_pc_borough_time)
    else:
        tp_pc_borough_time_df = tp_pc_borough_time

    #if trips_over_time_df is not None:
    #    trips_over_time_df = trips_over_time_df.union(trips_over_time)
    #else:
    #    trips_over_time_df = trips_over_time
    #break

# Define the file paths in GCS to save the DataFrames
#file_path_tp_pc_borough_time = 'gs://my-bigdata-project-cm/aggregated/tp_pc_borough_time.parquet'
#file_path_trips_over_time = 'gs://my-bigdata-project-cm/aggregated/trips_over_time.parquet'

# Save the DataFrames to Parquet format
#tp_pc_borough_time_df.write.parquet(file_path_tp_pc_borough_time, mode='overwrite')
#trips_over_time_df.write.parquet(file_path_trips_over_time, mode='overwrite')

# Display the first few rows of the Pandas DataFrame
print(f'tp_pc_borough_time_df:{tp_pc_borough_time_df.count()}')
print(tp_pc_borough_time_df.show(5))
#print(f'trips_over_time_df:{trips_over_time_df.count()}')
#print(trips_over_time_df.show(5))

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x7f5bdfe06990>)

# Iterate over each column in tp_pc_borough_time_df
for col_name in tp_pc_borough_time_df.columns:
    # Find the distinct values in the column
    distinct_values = tp_pc_borough_time_df.select(col_name).distinct().collect()
    # Print the column name and its distinct values
    print(f"Unique values in column '{col_name}':")
    for row in distinct_values:
        print(row[0])


# Define the file paths in GCS to save the DataFrames
file_path_tp_pc_borough_time = 'gs://my-bigdata-project-cm/aggregated/tp_pc_borough_time.parquet'
file_path_trips_over_time = 'gs://my-bigdata-project-cm/aggregated/trips_over_time.parquet'

# Save the DataFrames to Parquet format
tp_pc_borough_time_df.write.parquet(file_path_tp_pc_borough_time, mode='overwrite')
trips_over_time_df.write.parquet(file_path_trips_over_time, mode='overwrite')
